In [4]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt > shakespeare.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  4029k      0 --:--:-- --:--:-- --:--:-- 4064k


In [2]:
# read it in to inspect it
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


#### Simple Tokenizer using char->int mappings

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)} # simple mapping from set of unique chars
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]  # encode function to lookup the stoi dict
decode = lambda i: ''.join([itos[x] for x in i])  #decode func to lookup the itos dict

print(encode('hello'))
print(decode(encode('hello ')))  # 2-way check

[46, 43, 50, 50, 53]
hello 


#### Encode the entire dataset, using our tokenizer.

In [11]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
# Also split the dataset into train-val
n = int(0.9 * len(data))  # 90-10 split
train_data = data[:n]
val_data = data[n:]


#### Batching of dataset

In [13]:
# The dataset is divided on basis of 2 dimensions. Time Dimension and Batch Size.
# Time dimension is basically the context length. How many tokens our model sees before it predicts the next token.
# Karpathy says it as block_size.
# The Batch Dimension or Batch size is just no. of independent sequences the model will train on parallely(right word?)
# basically the batch size in a mini batch gradient descent, nothing else.

block_size = 8
batch_size = 4

def get_batch(split):
    # generate a mini batch of inputs, targets
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - batch_size, (batch_size,))
    x = torch.stack([data[i:block_size+i] for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    return x,y

xb, yb = get_batch('train')

print("inputs:")
print(xb.shape)
print(xb)
print("\ntargets:")
print(yb.shape)
print(yb)

# The logic behind these inputs-targets is that elements in target array, are the next token of the elements in inputs.
# visualization
print('---------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,: t+1]
        target = yb[b,t]

        print(f'when input is {context.tolist()}, target is {target}')


inputs:
torch.Size([4, 8])
tensor([[60, 43, 56, 63,  1, 56, 53, 53],
        [ 1, 39, 50, 50,  1, 61, 47, 58],
        [ 7, 40, 56, 43, 43, 42, 47, 52],
        [47, 45, 46, 40, 53, 59, 56,  1]])

targets:
torch.Size([4, 8])
tensor([[43, 56, 63,  1, 56, 53, 53, 58],
        [39, 50, 50,  1, 61, 47, 58, 46],
        [40, 56, 43, 43, 42, 47, 52, 45],
        [45, 46, 40, 53, 59, 56,  1, 51]])
---------------
when input is [60], target is 43
when input is [60, 43], target is 56
when input is [60, 43, 56], target is 63
when input is [60, 43, 56, 63], target is 1
when input is [60, 43, 56, 63, 1], target is 56
when input is [60, 43, 56, 63, 1, 56], target is 53
when input is [60, 43, 56, 63, 1, 56, 53], target is 53
when input is [60, 43, 56, 63, 1, 56, 53, 53], target is 58
when input is [1], target is 39
when input is [1, 39], target is 50
when input is [1, 39, 50], target is 50
when input is [1, 39, 50, 50], target is 1
when input is [1, 39, 50, 50, 1], target is 61
when input is [1, 39,

Simplest Language Model: Bigram LM.
1. works with pairs of characters next to each other
2. learns the probability distribution of which character has higher probability of appearing with the given character


In [82]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # directly reads off the probability for next token 
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=10)[0].tolist()))

torch.Size([256, 65])
tensor(4.6581, grad_fn=<NllLossBackward0>)

Kh-SssOQrk


In [84]:
# Training the model
optim = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for _ in range(10000):
    xb, yb = get_batch('train')
    # get the output and the loss
    logits, loss = m(xb,yb)

    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()
print(loss.item())

2.361586093902588


In [93]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Hay t yous IKI inoaqusen makeses I'd beuce?
S:
Bowir hovesthtonge m hed cithithavoif f bs, ve, gure;


In [94]:
torch.triu(torch.ones(3,3))

tensor([[1., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])

In [132]:
B,T,C = 4,8,2 # batch, time, channels
# imagine batch as a single sentence
# time defines the number of tokens in the sentence
# channels define the data/embeddings belonging to the actual token.
x = torch.randn(B,T,C)

# self-attention, finally
# single head of attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x)  # (B, T, 16)
v = value(x)

weights = q @ k.transpose(-2,-1)  # (B, T, 16) * (B, 16, T) -> (B, T, T)

# weighted aggregation trick. creating a lower triangle of 1s, and matmul with a matrix will give us sort of a mask,
# that doesnt allow the model to see the future tokens.
tril = torch.tril(torch.ones(T,T))
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights,dim=-1)

output = weights @ v

output




tensor([[[-7.8009e-01,  2.0357e-01,  8.8040e-01,  5.2806e-01,  2.8297e-01,
           8.6944e-01, -2.2371e-01, -3.7576e-01, -7.6070e-01,  5.0455e-01,
           1.9049e-01, -6.4040e-02,  1.0003e-01,  6.8179e-01,  4.3075e-01,
           4.9797e-02],
         [-2.8999e-01,  1.2600e-01,  2.4025e-01,  2.4115e-01,  1.6343e-01,
           3.3082e-01, -1.3248e-01, -9.1101e-02, -3.1777e-01,  1.1215e-01,
           1.0715e-01,  3.3474e-02,  9.9169e-03,  1.8785e-01,  1.7111e-01,
           1.1418e-02],
         [-5.1117e-01,  4.2295e-01,  7.6118e-02,  6.0407e-01,  5.2052e-01,
           6.1353e-01, -4.3035e-01,  3.3322e-02, -6.9974e-01, -1.0325e-01,
           3.3389e-01,  2.8762e-01, -9.1352e-02,  6.9303e-02,  3.4544e-01,
          -8.1848e-03],
         [-3.9075e-01,  4.3407e-01, -1.3337e-01,  5.6046e-01,  5.2607e-01,
           4.8576e-01, -4.3751e-01,  1.3240e-01, -6.1188e-01, -2.4489e-01,
           3.3520e-01,  3.4593e-01, -1.2985e-01, -9.1402e-02,  2.8822e-01,
          -2.1869e-02],
    

In [2]:
import torch
import torch.nn as nn

vocab_size = 50257
block_size = 256
n_embed = 384
n_head = 16

B,T = 2,8
x = torch.randint(0,vocab_size, (B,T))
pos = torch.arange(0, T, dtype = torch.long)

wte = nn.Embedding(vocab_size, n_embed)
wpe = nn.Embedding(block_size, n_embed)
c_attn_kv = nn.Linear(n_embed, 2 * n_embed)
c_attn_q = nn.Linear(n_embed, n_embed)

y = wpe(pos) + wte(x)

B, T, C = y.shape

kv = c_attn_kv(y)
q = c_attn_q(y)

# k,v = kv.split(n_embed, dim=2)


# q.view(B, T, n_head, C // n_head).shape

In [4]:
print(f'{32768 * 5000:e}') 

1.638400e+08
